<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_0618/blob/main/tibame_0709_ptt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
403 Forbidden
1. IP ban: 換ip
2. 模仿的不像: 少了必要的Headers

```



In [30]:
import urllib.request as req
# 相對路徑
# 1. a.txt: 現在目錄下的a.txt
# 2. ./a.txt: 現在目錄下的a.txt
# 3. ../a.txt: 前一層目錄下的a.txt

def download_imgur_image(url, dirname="."):
    # url = "https://i.imgur.com/OSwXeuz.jpeg"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
    resp = req.urlopen(r)
    content = resp.read()
    # 純文字檔案(CSV, txt...): open("a.txt", "w/r", encoding="utf-8")
    # 非純文字檔案(mp3, mp4, pdf, docs....): open("a.jpeg", "wb/rb")
    fp = dirname + "/" + url.split("/")[-1]
    with open(fp, "wb") as f:
        f.write(content)

url = "https://i.imgur.com/OSwXeuz.jpeg"
download_imgur_image(url)

In [32]:
import urllib.request as req
import json
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
# print(html)

metas = html.find_all("span", {"class":"article-meta-value"})
author = metas[0]
board = metas[1]
title = metas[2]
post_time = metas[3]
push_datas = html.find_all("div", {"class":"push"})


author_text = author.text.strip()
board_text = board.text.strip()
title_text = title.text.strip()
post_time_text = post_time.text.strip()

pushes = []
for p in push_datas:
    push_meta = p.find_all("span")
    # extract
    push_tag = push_meta[0]
    push_uid = push_meta[1]
    push_content = push_meta[2]
    push_ipdatetime = push_meta[3]
    trans = {"推":1, "噓":-1, "→":0}
    # text
    push_data = {
        "push_tag":trans[push_tag.text.strip()],
        "push_uid":push_uid.text.strip(),
        "push_content":push_content.text.strip().replace(": ", ""),
        "push_ipdatetime":push_ipdatetime.text.strip()
    }
    pushes.append(push_data)

# downlaod image
allow = ["png", "jpg", "jpeg", "gif", "webp"]
links = html.find_all("a")
for link in links:
    href = link["href"]
    sub = href.split(".")[-1]
    if sub.lower() in allow:
        download_imgur_image(href)

# 區塊.extract(): 在整份html裡消失
main_content = html.find("div", {"id":"main-content"})
for e in main_content.find_all("div", {"class":"article-metaline"}):
    e.extract()
for e in main_content.find_all("div", {"class":"article-metaline-right"}):
    e.extract()
for e in main_content.find_all("div", {"class":"push"}):
    e.extract()
content_text = main_content.text

row = {
    "author":author_text,
    "board":board_text,
    "title":title_text,
    "post_time":post_time_text,
    "content":content_text,
    "pushes":pushes,
}
# print(row)


fp = url.split("/")[-1] + ".json"
f = open(fp, "w", encoding="utf-8")
json.dump(row, f, indent=4, ensure_ascii=False)
f.close()